# Environment Setup

In [ ]:
! pip install -Uqq fastbook waterfallcharts treeinterpreter dtreeviz==1.4.1
import warnings
warnings.filterwarnings("ignore")
import os
import numpy
import pandas
import fastbook
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from treeinterpreter import treeinterpreter
from sklearn.model_selection import cross_val_score
from sklearn.inspection import PartialDependenceDisplay
fastbook.setup_book()

# Data Import & Exploration

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df_raw = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
print("df Shape: ",df_raw.shape)
print(df_raw.columns)
df_raw.describe()

# Random Forest with SciKit

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = df_raw.copy()
y = np.log(df['SalePrice'])
X = df.drop('SalePrice',axis=1)
print("Total Features:",len(X.columns))
numeric_features = X.select_dtypes(include=['number']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
print("Numeric:", len(numeric_features),"Categorical",len(categorical_features))

## Hyperparameter Tuning

In [ ]:
numeric_transformer = SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

X_train, X_valid, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_valid)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

#Gradient Boosting
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(n_estimators=100, random_state=42))
])


X_train, x_valid, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_valid)


mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE) with Gradient Boosting: {rmse}")

# Random Forest

In [ ]:
param_dist = {
    'model__n_estimators': [100, 200, 300, 400],
    'model__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'model__max_depth': [3, 4, 5, 6],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4],
    'model__subsample': [0.8, 0.9, 1.0]
}

random_search = RandomizedSearchCV(
    estimator=model_pipeline,
    param_distributions=param_dist,
    n_iter=20,  
    cv=5,  
    verbose=1,
    random_state=42,
    n_jobs=-1,  
    scoring='neg_mean_squared_error'
)

random_search.fit(X_train, y_train)


best_model = random_search.best_estimator_
print(f"Best parameters found: {random_search.best_params_}")

y_pred = best_model.predict(X_valid)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE) with tuned Gradient Boosting: {rmse}")

In [ ]:
X_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
y_pred_test = best_model.predict(X_test)
submission = pd.DataFrame({
    'Id': X_test['Id'], 
    'SalePrice': np.exp(y_pred_test)
})

submission.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' created successfully.")

## Preprocessing

In [ ]:
df = df_raw.copy()
df['SalePrice'] = np.log(df['SalePrice'])
procs = [Categorify, FillMissing]
cont,cat = cont_cat_split(df, 1, dep_var='SalePrice')
splits = RandomSplitter(valid_pct=0.3, seed=42)(range_of(df))
print("Continuous:",len(cont), "Categorical:",len(cat))

In [ ]:
dls = TabularDataLoaders.from_df(df, procs=procs, cat_names=cat, cont_names=cont, y_names='SalePrice', splits=splits, bs=64)
print("Train:", len(dls.train.items), "Valid:", len(dls.valid.items))
dls.show(3)
dls.items.head(3)

## Base Model

In [ ]:
def rf(xs, y, n_estimators=250, max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators, max_features=max_features,
                                min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)


In [ ]:
xs, y, valid_xs, valid_y = dls.train.xs, dls.train.y, dls.valid.xs, dls.valid.y
m = rf(xs, y)

## Results

In [ ]:
def r_mse(pred,y): 
    return round(math.sqrt(((pred-y)**2).mean()), 6)

def m_rmse(m, xs, y): 
    return r_mse(m.predict(xs), y)

def evaluate_cross_val(m, xs, y):
    scores = cross_val_score(m, xs, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-scores)
    print("Cross-validation RMSE scores:", rmse_scores)
    print("Mean RMSE:", rmse_scores.mean())
    print("Standard deviation of RMSE:", rmse_scores.std())
    
def evaluate_model(m, xs, y, valid_xs, valid_y):
    print("Training Set RMSE:",m_rmse(m, xs, y))
    evaluate_cross_val(m, xs, y)
    print("_______________________________________________________________")
    print("OOB Error:",r_mse(m.oob_prediction_, y))
    print("Validation Set RMSE:",m_rmse(m, valid_xs, valid_y))

In [ ]:
evaluate_model(m, xs, y, valid_xs, valid_y)

## Base Model Imputation

## Feature Engineering

### Evaluate Feature Importance

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

In [ ]:
fi[fi['cols'] == 'SaleCondition']

In [ ]:
fi = rf_feat_importance(m, xs)

plot_fi(fi[:21]);

In [ ]:
to_keep = fi[fi.imp>0.0045].cols
len(to_keep)

In [ ]:
xs = xs[to_keep]
valid_xs = valid_xs[to_keep]
m = rf(xs, y)

In [ ]:
evaluate_model(m, xs, y, valid_xs, valid_y)

### Investigate Most Important Features

#### OverallQual (Overall material and finish quality)
Changes Attempted:
1. Change Cont - Cat
2. Square value
3. Custom bins


In [ ]:
df = df_raw.copy()
df['SalePrice'] = np.log(df['SalePrice'])
df['OverallQual_cat'] = pd.cut(df['OverallQual'], bins=[0,4,7,10], labels=['Poor', 'Ok','Great'])
procs = [Categorify, FillMissing]
cont,cat = cont_cat_split(df, 1, dep_var='SalePrice')
splits = RandomSplitter(valid_pct=0.2, seed=42)(range_of(df))
dls = TabularDataLoaders.from_df(df, procs=procs, cat_names=cat, cont_names=cont, y_names='SalePrice', splits=splits, bs=64)
xs, y, valid_xs, valid_y = dls.train.xs, dls.train.y, dls.valid.xs, dls.valid.y
m = rf(xs, y)
evaluate_model(m, xs, y, valid_xs, valid_y)

In [ ]:
df['OverallQual'].hist()
plt.show()

In [ ]:
PartialDependenceDisplay.from_estimator(m, valid_xs, ['OverallQual']);

#### GrLivingArea (Above ground square footage)

In [ ]:
df['GrLivArea'].min(), df['GrLivArea'].max()

In [ ]:
df['GrLivArea'].hist(bins=30)
plt.show()

In [ ]:
PartialDependenceDisplay.from_estimator(m, valid_xs, ['GrLivArea']);

### Year Built

In [ ]:
df['YearBuilt'].hist(bins= 20)
plt.show();

In [ ]:
PartialDependenceDisplay.from_estimator(m, valid_xs, ['YearBuilt','YearRemodAdd']);

In [ ]:
print(df['YearRemodAdd'].max(), df['YearRemodAdd'].min())
print(df['YearRemodAdd'][df['YearRemodAdd'].isnull()])
df['YearRemodAdd'].hist(bins= 50)
plt.show();

In [ ]:
df = df_raw.copy()
df['SalePrice'] = np.log(df['SalePrice'])
df['Score'] = (df['OverallQual']*df['OverallCond'])
procs = [Categorify, FillMissing]
cont,cat = cont_cat_split(df, 1, dep_var='SalePrice')
splits = RandomSplitter(valid_pct=0.2, seed=42)(range_of(df))
dls = TabularDataLoaders.from_df(df, procs=procs, cat_names=cat, cont_names=cont, y_names='SalePrice', splits=splits, bs=64)
xs, y, valid_xs, valid_y = dls.train.xs, dls.train.y, dls.valid.xs, dls.valid.y
xs = xs[to_keep]
valid_xs = valid_xs[to_keep]
m = rf(xs, y)
evaluate_model(m, xs, y, valid_xs, valid_y)

In [ ]:
df_raw.groupby('SaleCondition')['SalePrice'].mean().reset_index()

In [ ]:
df_raw.groupby('YrSold').agg(
    avg_column2=('SalePrice', 'mean'),
    count_column2=('Id', 'size')
).reset_index()

In [ ]:
df_test.groupby('SaleCondition').agg(
#     avg_column2=('SalePrice', 'mean'),
    count_column2=('Id', 'size')
).reset_index()

## Feature Scaling

In [ ]:
df = df_raw.copy()
df['SalePrice'] = np.log(df['SalePrice'])
df['GrLivArea'] = np.log(df['GrLivArea'])
df['YearBuilt'] = df['YearBuilt'] - df['YrSold']

In [ ]:
procs = [Categorify, FillMissing]
cont,cat = cont_cat_split(df, 1, dep_var='SalePrice')
cont.remove('OverallQual')
cat.append('OverallQual')
splits = RandomSplitter(valid_pct=0.2, seed=42)(range_of(df))
dls = TabularDataLoaders.from_df(df, procs=procs, cat_names=cat, cont_names=cont, y_names='SalePrice', splits=splits, bs=64)
xs, y, valid_xs, valid_y = dls.train.xs, dls.train.y, dls.valid.xs, dls.valid.y
m = rf(xs, y, n_estimators=500)
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)

In [ ]:
scores = cross_val_score(m, xs, y, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)

print("Cross-validation RMSE scores:", rmse_scores)
print("Mean RMSE:", rmse_scores.mean())
print("Standard deviation of RMSE:", rmse_scores.std())

## Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

df2 = df.copy()
imputer = KNNImputer(n_neighbors=5)
df2 = pd.get_dummies(df2, drop_first=True)
df2 = pd.DataFrame(imputer.fit_transform(df2), columns=df2.columns)

X = df2.drop('SalePrice', axis=1)
y = df2['SalePrice']




X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Define the hyperparameter grid
param_distributions = {
    'n_estimators': [100, 200, 500, 1000],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Step 4: Set up the model and RandomizedSearchCV
rf = RandomForestRegressor(random_state=42)

# Use RandomizedSearchCV for faster tuning
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=50,  # Number of parameter combinations to try
    scoring='neg_mean_squared_error',
    cv=3,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Step 5: Fit the model to the training data
random_search.fit(X_train, y_train)

# Step 6: Get the best hyperparameters
print(f"Best Parameters: {random_search.best_params_}")

# Step 7: Evaluate the model on validation set
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Validation RMSE: {rmse}")



# Step 8: Evaluate the model's performance

# Calculate performance metrics
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"Validation RMSE: {rmse}")
print(f"Validation MAE: {mae}")
print(f"Validation R²: {r2}")

# Step 9: Analyze Feature Importance

# Get feature importance
importances = best_rf.feature_importances_
feature_names = X.columns

# Create a DataFrame for better visualization
importances_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Print top features
print("Top 10 Most Important Features:")
print(importances_df.head(10))

# Plot feature importance
importances_df.head(10).plot(kind='barh', x='Feature', y='Importance', legend=False)
plt.title('Top 10 Most Important Features')
plt.show()

# Step 10: Residual Analysis

# Calculate residuals
residuals = y_val - y_pred

# Plot residuals
plt.scatter(y_pred, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

## Results

In [ ]:
# plot_fi(rf_feat_importance(m_imp, xs_imp));

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# from scipy.cluster.hierarchy import linkage, leaves_list

# def cluster_and_visualize_columns(df):
#     # Step 1: Compute correlation matrix of the columns
#     corr = df.corr()

#     # Step 2: Perform hierarchical clustering on the correlation matrix
#     Z = linkage(corr, method='ward')

#     # Step 3: Get the column order based on the clustering
#     col_order = leaves_list(Z)

#     # Step 4: Reorder the correlation matrix
#     clustered_corr = corr.iloc[col_order, col_order]

#     # Step 5: Plot the clustered correlation matrix with better spacing
#     plt.figure(figsize=(10, 8))  # Adjust figure size here
#     sns.heatmap(clustered_corr, annot=True, cmap='coolwarm', fmt='.2f', annot_kws={"size": 10},  # Adjust font size
#                 cbar_kws={'shrink': 0.75})  # Adjust color bar size
#     plt.xticks(rotation=45, ha='right', fontsize=10)  # Rotate x-axis labels
#     plt.yticks(fontsize=10)  # Adjust y-axis labels font size
#     plt.title("Clustered Correlation Matrix", fontsize=15)
#     plt.tight_layout()  # Ensure everything fits without overlap
#     plt.show()

# cluster_and_visualize_columns(xs_imp)

In [ ]:
# import warnings
# warnings.simplefilter('ignore', FutureWarning)

# from treeinterpreter import treeinterpreter
# from waterfall_chart import plot as waterfall

In [ ]:
# row = valid_xs_imp.iloc[:5]

In [ ]:
# prediction,bias,contributions = treeinterpreter.predict(m_imp, row.values)

In [ ]:
# prediction[0], bias[0], contributions[0].sum()

In [ ]:
# waterfall(valid_xs_imp.columns, contributions[0], threshold=0.08, 
#           rotation_value=45,formatting='{:,.3f}');

## Submission

In [ ]:
# df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv").fillna(0)

In [ ]:
# df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv").fillna(0)
# df_test['GrLivArea'] = np.log(df['GrLivArea'])
# dls_test = dls.test_dl(df_test)
# xs_test = dls_test.xs
# predictions,bias,contributions = treeinterpreter.predict(m, xs_test)
# final_preds = np.exp(predictions)
# test_ids = np.array(df_test['Id'])
# submission_df = pd.DataFrame({
#     'Id': test_ids,
#     'SalePrice': final_preds[:, 0]
# })

# submission_df.to_csv('/kaggle/working/submission.csv', index=False)

# Model Improvement & Iteration

## 1. Impact of Financial Crisis

In [ ]:
# priceYear = df.loc[:,['YrSold','SalePrice']]
# avgPriceYear = priceYear.groupby('YrSold')['SalePrice'].mean()
# avgPriceYear

Its pretty clear that there was a break in trend around 2008, which coincides with the financial crisis and subsequent housing crash. Below, I will try to figure out exactly which month this break occurred.

### Partial Dependence

In [ ]:
# from sklearn.inspection import PartialDependenceDisplay
# PartialDependenceDisplay.from_estimator(m, valid_xs, ['YrSold','MoSold'])